# Use Tran$

In [3]:
import torch

In [4]:
# add to $y$tem path the parent directory
import os
import sys
notebook_dir = os.path.abspath(os.path.dirname(''))
parent_dir = os.path.abspath(os.path.join(notebook_dir, os.pardir))
sys.path.append(parent_dir)

In [5]:
from transformer import model
import torch

F:\SpellChecker\venv\lib\site-packages\torch\nn\modules\transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
from transformer import transformer_probab,transformer_probab_list,transformer_probab_final_word

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [1]:
# model.load_state_dict(torch.load('models/transformer_model.pt'))

In [2]:
sample_sentences = ['हरेक [MASK] नेपामको संविधानक पालना गर्नुपर्छ ।' ,
                    'म पुस्तकालयबाट थुलो किताब पढ्न चाहन्छ ।',
                    'तर उस समयमा पनि स्वस्थ राजनैतिक वातावरनको अभावले गर्दा देश विकासतर्फ विशेष प्रगति हुन  सकेन।',
                   'नेपालमा आधुनिक रुपमा आर्थक विकाससम्बन्धी कार्यरू प्रारम्भ भएको हालै मात्र हो।',
                   'हार धुनुहोस् र स्वास्थ जीवन जिउनुहोस्।',
                   'जब प्रवीधिहरू एकीकृत हुन सूरु गर्छन् अर्थतन्त्र तथ सँस्कृति पनि निश्चितरूपमा विस्तारै एकीकृत हुने छ।',
                   'उद्देयहरुमा पनि कुनै एक उद्देश्य पूर्ति नहुँदै अर्को नयाँ  उद्देश्यको रुपमा लिइने परम्परा बस्यो।',
                   'लगानीकर्ताहरूको धयान तुरुन्त फेरियो , व्यापारीहरुले वताए ।',
                    'अति धेरै हिज्जे गलती भएका शब्दहरू । तपाईँले टाइप गर्दा हिज्जे जाँच अक्षम पारयो ।',
                    'लामो',
                    'म नेपाली राम्रोसँग बोल्दिन',
                    'तपाईंको उमर कति हो?',
                    'मलाइ एक्लै छोडनुहोस्',
                   'रुसी राष्ट्रपती पुटिनको प्रेम जावन त्य्ती सफल छैन ।']

In [7]:
#help(tokenizer)

# Functions to correct

In [7]:
import torch
import pickle
import BrillMoore
import regex as re
from utils import final_candidate_words, return_lexicon_dict
import math
import itertools

def words(text): 
    text = re.sub(r'[\u0964]', r'\u0020\u0964\u0020', text)
    return re.findall(r'[\u0900-\u097F]+', text.lower())

final_lexicon_dict = return_lexicon_dict()

with open(os.path.join(notebook_dir, '..','models','bma_27dec.pickle'),'rb') as f:
    bma = pickle.load(f)

In [8]:
def likelihood_bm_word(word, candidate_word):
    
    prod = bma.likelihood(word, candidate_word)

    return prod


alpha = 0.65
def constant_distributive_likelihood_word(word,candidate_word,candidate_count):
    prod = 1    

    if word==candidate_word:
        prod*= alpha
    else:
        prod*= (1-alpha)/candidate_count
        
    return prod

In [9]:
def correct_current_word(sentence, model,p_lambda = 1,l_lambda = 1,prior='transformer',trie = False,likelihood = 'default',top = 2,return_prob = False):
# Attempts to correct only final word of the given sentence

    tokens = words(sentence)
    final_token = tokens[-1]
    candidates = []
    
#     print(tokens)
    #Append the candidates of the final word
    candidates.append(final_candidate_words(tokens[-1],use_trie = trie,force = True))
    
#     print(candidates)
    
    if len(tokens) > 1 and len(candidates[0])>1:
        
        if likelihood=='default':
            candidate_probab = transformer_probab_final_word([sentence],candidates = candidates)
            candidate_count = len(candidates[0])
            sentences_probab_post=[(row*p_lambda) +
                                   (math.log(constant_distributive_likelihood_word(final_token,candidate_token,candidate_count))*l_lambda) 
                                   for row,candidate_token in zip(candidate_probab,candidates[0])]
        elif likelihood=='bm':
            candidate_probab = transformer_probab_final_word([sentence],candidates = candidates)
            modifier_probab_dict = {w:(-1.5 if (w not in final_lexicon_dict) else 1.0) for p,w in zip(candidate_probab,candidates[0])}
            sentences_probab_post=[(row*p_lambda) + 
                                    modifier_probab_dict[candidate_token]+ math.log(likelihood_bm_word(final_token,candidate_token))*l_lambda
                                    for row,candidate_token in zip(candidate_probab,candidates[0])]
        elif likelihood =='bmWithNoLM':
            
            modifier_probab_dict = {w:(-1.5 if (w not in final_lexicon_dict) else 1.0) for w in candidates[0]}
            sentences_probab_post=[modifier_probab_dict[candidate_token]+math.log(likelihood_bm_word(final_token,candidate_token))*l_lambda
                                    for candidate_token in candidates[0]]
    
        elif likelihood =='defaultWithNoLM':
            
            modifier_probab_dict = {w:(-1.5 if (w not in final_lexicon_dict) else 1.0) for w in candidates[0]}
            candidate_count = len(candidates[0])
            sentences_probab_post=[modifier_probab_dict[candidate_token]+math.log(constant_distributive_likelihood_word(final_token,candidate_token,candidate_count))*l_lambda
                                    for candidate_token in candidates[0]]
            
        # Evaluate the probabilies of the final word given context
        
        
        sorted_indices_desc = sorted(range(len(sentences_probab_post)), key=lambda k: sentences_probab_post[k], reverse=True)
        
        return_candidates = [candidates[0][i] if return_prob==False else (candidates[0][i], sentences_probab_post[i])for n,i in enumerate(sorted_indices_desc)]
#         print(candidates, sentences_probab_post,return_candidates)
        
        if len(return_candidates)>top:
            return return_candidates[:top]
        
        return return_candidates
     
    
    return ([tokens[-1]]+candidates[0])


def extract_choices_word(sentence, model,p_lambda = 1,l_lambda = 1,prior='transformer',trie = False,likelihood = 'default',top = 6):
    all_candidates = []
    tokens = words(sentence)
    
    
    for i in range(len(tokens)):
        c = correct_current_word(sentence = ' '.join(tokens[:i+1]),model = model,p_lambda = p_lambda,l_lambda = l_lambda,prior=prior,trie = trie,likelihood =likelihood,top = top,return_prob = False)
        all_candidates.append(c)
        
    return all_candidates

def autocorrect_word(sentence, model,p_lambda = 1,l_lambda = 1,prior='transformer',trie = False,likelihood = 'default',top = 6):
    all_candidates = []
    tokens = words(sentence)
    
    
    for i in range(len(tokens)):
        s = tokens[:i+1]
        c = correct_current_word(sentence = ' '.join(tokens[:i+1]),model = model,p_lambda = p_lambda,l_lambda = l_lambda,prior=prior,trie = trie,likelihood =likelihood,top = top,return_prob = True)
        
        if len(c) > 1 and type(c[0]) != str :
            temp = [c[0][0]]
            tokens[i] = c[0][0]
            for w in range(len(c)-1):
                if c[0][1] - c[w+1][1] < 2.00:
                    temp.append(c[w+1][0])
                    
            all_candidates.append(temp)
            continue
        all_candidates.append(c)
        
    return all_candidates

In [12]:
extract_choices_word(sample_sentences[1],model=model,p_lambda = 1.2,trie = True,likelihood = 'bm')

[['म', 'म', 'मा', 'अ', 'आ', 'आम'],
 ['पुस्तकालयबाट', 'पुस्तकालयमा'],
 ['ठूलो', 'थलो', 'ठुलो', 'ठुला', 'थुलो'],
 ['किताब', 'किताबमा', 'किनार', 'कतार', 'किटान'],
 ['पढ्न', 'पढ्ने', 'चढ्न', 'पढ्दा', 'बढ्न'],
 ['चाहन्छु', 'चाहन्छ', 'चाहन्छन्', 'चाहिन्छ', 'चाहना'],
 ['।', 'अ', 'इ', 'ः', 'आ']]

In [ ]:
sample_sentences[2]

In [14]:
extract_choices_word(sample_sentences[2],model=model,p_lambda = 1.2,trie = True,likelihood = 'bm')

[['तर', 'तार', 'आर', 'कर', 'गर'],
 ['उस', 'उसो', 'आस', 'उप', 'एस'],
 ['समयमा', 'समयमै', 'समयका', 'सडकमा', 'संघमा'],
 ['पनि', 'पति', 'नि', 'उनि', 'अनि'],
 ['स्वस्थ', 'अस्वस्थ', 'स्वस्थ्य', 'ध्वस्त', 'स्वच्छ'],
 ['राजनैतिक', 'राजनीतिक', 'राजनीतिको', 'राजनीति', 'राजनीतिका'],
 ['वातावरणको', 'वातावरनको'],
 ['अभावले', 'अभावमा', 'अभावका', 'अभावको', 'सभाले'],
 ['गर्दा', 'गर्दै', 'गर्न', 'गर्ला', 'गर्व'],
 ['देश', 'दश', 'आदेश', 'देव', 'पेश'],
 ['विकासतर्फ'],
 ['विशेष', 'बिशेष', 'विदेश', 'विशेषत', 'विशेषता'],
 ['प्रगति', 'प्रति', 'प्रगतिको', 'प्रगाढ', 'प्रालि'],
 ['हुन', 'जुन', 'कुन', 'छुन', 'धुन'],
 ['सकेन', 'सकेका', 'सकेनन्', 'सकिन', 'सकिएन'],
 ['।', 'ः', 'इ', 'अ', 'आ']]

In [15]:
autocorrect_word(sample_sentences[2], model,p_lambda = 1.2,l_lambda = 1,likelihood = 'bm',prior='transformer',trie = True,top = 3)

[['तर', 'तार', 'आर', 'कर', 'गर'],
 ['उस'],
 ['समयमा'],
 ['पनि'],
 ['स्वस्थ'],
 ['राजनैतिक', 'राजनीतिक'],
 ['वातावरणको'],
 ['अभावले'],
 ['गर्दा'],
 ['देश', 'दश'],
 ['विकासतर्फ'],
 ['विशेष'],
 ['प्रगति'],
 ['हुन'],
 ['सकेन'],
 ['।']]

# Evaluation

In [10]:
from eval import gather_dataset,WER,word_accuracy,char_accuracy

dataset_file = os.path.join(notebook_dir, '..','data','eval_data2.pic')

In [11]:
dataset = gather_dataset(dataset_file)[:400]

In [12]:
correct_tokens = [words(t[0]) for t in dataset]
error_tokens = [words(t[1]) for t in dataset]
error_sentences = [t[1] for t in dataset] 


### Bm

In [16]:
Gap = 0.2
a = 10
def find_p_lambda(error_sentences):
    
    for j in range(8):
        predicted_tokens = []
        for i,s in enumerate(error_sentences[:a]):
            c = autocorrect_word(s, model,p_lambda = 0 + Gap*j,l_lambda = 1,likelihood = 'bm',prior='transformer',trie = True,top = 2)
        #     print(c)
            c = [t[0] for t in c]
            predicted_tokens.append(c)
            if i%2 == 0:
                print(i)
#         word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a])
        print(Gap*j,' : ',word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a]),'WER1: ',WER(correct_tokens[:a],error_tokens[:a] ),'WER2: ',WER(correct_tokens[:a],predicted_tokens[:a] ))
find_p_lambda(error_sentences)

0
2
4
6
8
0.0  :  (0.42857142857142855, 18, 42) WER1:  0.2727272727272727 WER2:  0.19480519480519481
0
2
4
6
8
0.2  :  (0.6190476190476191, 26, 42) WER1:  0.2727272727272727 WER2:  0.14935064935064934
0
2
4
6
8
0.4  :  (0.6904761904761905, 29, 42) WER1:  0.2727272727272727 WER2:  0.12987012987012986
0
2
4
6
8
0.6000000000000001  :  (0.7142857142857143, 30, 42) WER1:  0.2727272727272727 WER2:  0.12987012987012986
0
2
4
6
8
0.8  :  (0.7142857142857143, 30, 42) WER1:  0.2727272727272727 WER2:  0.13636363636363635
0
2
4
6
8
1.0  :  (0.7142857142857143, 30, 42) WER1:  0.2727272727272727 WER2:  0.14935064935064934
0
2
4
6
8
1.2000000000000002  :  (0.7142857142857143, 30, 42) WER1:  0.2727272727272727 WER2:  0.16233766233766234
0
2
4
6
8
1.4000000000000001  :  (0.6904761904761905, 29, 42) WER1:  0.2727272727272727 WER2:  0.17532467532467533


In [16]:
Gap = 0.2
a = 10
def find_p_lambda(error_sentences):
    
    for j in range(8):
        predicted_tokens = []
        for i,s in enumerate(error_sentences[:a]):
            c = autocorrect_word(s, model,p_lambda = 0 + Gap*j,l_lambda = 1,likelihood = 'bm',prior='transformer',trie = True,top = 2)
        #     print(c)
            c = [t[0] for t in c]
            predicted_tokens.append(c)
            if i%2 == 0:
                print(i)
#         word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a])
        print(Gap*j,' : ',word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a]),'WER1: ',WER(correct_tokens[:a],error_tokens[:a] ),'WER2: ',WER(correct_tokens[:a],predicted_tokens[:a] ))
find_p_lambda(error_sentences)

0
2
4
6
8
0.0  :  0.42857142857142855
0
2
4
6
8
0.2  :  0.6428571428571429
0
2
4
6
8
0.4  :  0.6666666666666666
0
2
4
6
8
0.6000000000000001  :  0.7142857142857143
0
2
4
6
8
0.8  :  0.7142857142857143
0
2
4
6
8
1.0  :  0.7142857142857143
0
2
4
6
8
1.2000000000000002  :  0.7142857142857143
0
2
4
6
8
1.4000000000000001  :  0.6904761904761905


In [17]:
##Use 1.2 a$ plambda value

In [16]:
predicted_tokens = []
for i,s in enumerate(error_sentences[:400]):
    c = autocorrect_word(s, model,p_lambda = 1.2,l_lambda = 1,likelihood = 'bm',prior='transformer',trie = True,top = 2)
#     print(c)
    c = [t[0] for t in c]
    torch.cuda.empty_cache()
    predicted_tokens.append(c)
    if i%2 == 0:
        print(i)
        

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacty of 6.00 GiB of which 0 bytes is free. Of the allocated memory 12.19 GiB is allocated by PyTorch, and 499.26 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [17]:
i

324

In [16]:
# def side_by_side(corrected_tokens, predicted_tokens):
comp = [[a+'--'+b+'--'+d for a,b,d in zip(c,e,p)] for c,e,p in zip(correct_tokens, error_tokens,predicted_tokens)]

In [17]:
comp

[['अब--अब--अब',
  'गाह्रो--गाह्रो--गाह्रो',
  'छैन--छैन--छैन',
  'प्रदूषण--प्रदूषीण--प्रदूषण',
  'नाप्न--नाप्न--नाप्न',
  'भारतीय--भारेतीय--भारतीय',
  'पूर्व--पूर्व--पूर्व',
  'प्रधानमन्त्रीलाई--ज्रधानमन्त्रीलाई--प्रधानमन्त्रीलाई',
  'उपप्रधानमन्त्री--उपप्रधानकन्त्री--उपप्रधानमन्त्री',
  'निधीले--निधीले--निधिले',
  'किन--किन--किन',
  'भेटे--भेटे--भेट',
  '।--।--।'],
 ['लाठी--लाठी--लाठी',
  'लिएका--लिएका--लिएका',
  'हजारौँ--हजारौँ--हजारौं',
  'संख्यामा--संख्यामा--संख्यामा',
  'रहेका--रथहेका--रहेका',
  'थारुहरुले--थारुहउरुले--थारुहरुले',
  'र्याली--र्याली--र्याली',
  'प्रर्दशन--प्रर्दशन--प्रर्दशन',
  'गरी--गरती--गरको',
  'राजापुर--नोजापुर--भोजपुर',
  'बजारको--बजिरको--बजारको',
  'दक्षिण--दक्षिण--दक्षिण',
  'चौकमा--चौकमा--चौकीमा',
  'कोणसभा--कोणनसभा--कोणसभा',
  'गरेका--गरेक--गरेका',
  'थिए--थिए--थिए',
  '।--।--।'],
 ['आफूहरुले--आफूहरुले--आफूहरुले',
  'आफनो--आफनो--आफ्नो',
  'स्वाभिमानको--स्वाभिमानमो--स्वाभिमानमो',
  'लडाइ--लडाइ--लडाइ',
  'लडेको--लडेमो--लडेको',
  'उनको--उनको--उनको',
  'भनाई-

In [15]:
# import pickle
# with open('corrected_tran_word.p','wb') as f:
#     pickle.dump(predicted_tokens,f)

In [27]:
torch.cuda.empty_cache()

In [18]:
def WER(correct_tokens,predicted_tokens):
    '''
    Calculates the Word error rate for given list of correct and predicted tokens
    
    '''
    
    assert len(correct_tokens)==len(predicted_tokens),"Dimension of correct and predicted tokens don't match"
    
    if type(correct_tokens[0]) == list or type(correct_tokens[0]) == tuple:
        correct_tokens = [w for s in correct_tokens for w in s]
        predicted_tokens = [w for s in predicted_tokens for w in s]
    
    if type(correct_tokens[0]) == str:
        total = len(correct_tokens)
        errors = 0
        for c,p in zip(correct_tokens,predicted_tokens):
            if c!=p:
                errors+=1
#                 print(errors)
        return errors/total

In [19]:
# with open('coorected_tran_word','rb') as f:
#     c = pickle.load(f)

In [18]:
a = len(predicted_tokens)
WER(correct_tokens[:a],error_tokens[:a] ),WER(correct_tokens[:a],predicted_tokens[:a] )

(0.25042111173498033, 0.16657308628111547)

In [19]:
word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a])

(0.6472346786248132, 866, 1338)

In [22]:
# from BrillMoore import BrillMore

# align = BrillMore().align
# def char_accuracy(correct_tokens,predicted_tokens,error_tokens):
#     '''
#     Calculates the Word accuracy rate for given list of correct,predicted and error tokens
    
#     '''
    
#     assert len(correct_tokens)==len(predicted_tokens),"Dimension of correct and predicted tokens don't match"
    
#     if type(correct_tokens[0]) == list or type(correct_tokens[0]) == tuple:
#         correct_tokens = [w for s in correct_tokens for w in s]
#         predicted_tokens = [w for s in predicted_tokens for w in s]
#         error_tokens = [w for s in error_tokens for w in s]
    
#     if type(correct_tokens[0]) == str:
#         corrected = 0
#         errors = 0
#         for c,p,e in zip(correct_tokens,predicted_tokens,error_tokens):
#             if c!=e:
                
#                 alignment1 = align(e,c)
#                 alignment2 = align(p,c)
#                 e = sum([True if (t[0]!=t[1]) or (None in t) else False for t in alignment1])
#                 errors+=e
#                 corrected += abs(sum([True if ((t[0]!=t[1]) or (None in t)) else False for t in alignment2])-e)
# #                 print(e,corrected)

#         return corrected/errors,corrected,errors


In [20]:
char_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a])

(0.6947969543147208, 1095, 1576)

### constant distributive likelihood

In [19]:
Gap = 0.2
a = 10
def find_p_lambda(error_sentences):
    
    for j in range(8):
        predicted_tokens = []
        for i,s in enumerate(error_sentences[:a]):
            c = autocorrect_word(s, model,p_lambda = 0 + Gap*j,l_lambda = 1,likelihood = 'default',prior='transformer',trie = True,top = 4)
        #     print(c)
            c = [t[0] for t in c]
            predicted_tokens.append(c)
            if i%2 == 0:
                print(i)
#         word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a])
        print(Gap*j,' : ',word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a]),'WER1: ',WER(correct_tokens[:a],error_tokens[:a] ),'WER2: ',WER(correct_tokens[:a],predicted_tokens[:a] ))
find_p_lambda(error_sentences)

0
2
4
6
8
0.0  :  0.0 WER1:  0.2727272727272727 WER2:  0.2727272727272727
0
2
4
6
8
0.2  :  0.5476190476190477 WER1:  0.2727272727272727 WER2:  0.12337662337662338
0
2
4
6
8
0.4  :  0.6190476190476191 WER1:  0.2727272727272727 WER2:  0.11038961038961038
0
2
4
6
8
0.6000000000000001  :  0.6666666666666666 WER1:  0.2727272727272727 WER2:  0.11688311688311688
0
2
4
6
8
0.8  :  0.6904761904761905 WER1:  0.2727272727272727 WER2:  0.17532467532467533
0
2
4
6
8
1.0  :  0.6666666666666666 WER1:  0.2727272727272727 WER2:  0.2012987012987013
0
2
4
6
8
1.2000000000000002  :  0.6666666666666666 WER1:  0.2727272727272727 WER2:  0.22727272727272727
0
2
4
6
8
1.4000000000000001  :  0.6428571428571429 WER1:  0.2727272727272727 WER2:  0.2532467532467532


In [20]:
#0.8

In [14]:
predicted_tokens = []
for i,s in enumerate(error_sentences):
    c = autocorrect_word(s, model,p_lambda = 0.8,l_lambda = 1,likelihood = 'default',prior='transformer',trie = True,top = 4)
#     print(c)
    c = [t[0] for t in c]
    torch.cuda.empty_cache()
    predicted_tokens.append(c)
    if i%2 == 0:
        print(i)
    if i==323:
        print(WER(correct_tokens[:a],error_tokens[:a] ),WER(correct_tokens[:a],predicted_tokens[:a] ),word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a]),char_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a]))
        

0
2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300
302
304
306
308
310
312
314
316
318
320
322


OutOfMemoryError: CUDA out of memory. Tried to allocate 18.00 MiB. GPU 0 has a total capacty of 6.00 GiB of which 0 bytes is free. Of the allocated memory 12.19 GiB is allocated by PyTorch, and 505.56 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Ablation study -word BM

In [41]:
Gap = 0.2
a = 10
def find_p_lambda(error_sentences):
    
    for j in range(8):
        predicted_tokens = []
        for i,s in enumerate(error_sentences[:a]):
            c = autocorrect_word(s, model,p_lambda = 0,l_lambda =  Gap*j,likelihood = 'bmWithNoLM',prior='transformer',trie = True,top = 4)
        #     print(c)
            c = [t[0] for t in c]
            predicted_tokens.append(c)
            if i%2 == 0:
                print(i)
#         word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a])
        print(Gap*j,' : ',word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a]),'WER1: ',WER(correct_tokens[:a],error_tokens[:a] ),'WER2: ',WER(correct_tokens[:a],predicted_tokens[:a] ))
find_p_lambda(error_sentences)

0
2
4
6
8
0.0  :  (0.5238095238095238, 22, 42) WER1:  0.2727272727272727 WER2:  0.4025974025974026
0
2
4
6
8
0.2  :  (0.5476190476190477, 23, 42) WER1:  0.2727272727272727 WER2:  0.2727272727272727
0
2
4
6
8
0.4  :  (0.5238095238095238, 22, 42) WER1:  0.2727272727272727 WER2:  0.19480519480519481
0
2
4
6
8
0.6000000000000001  :  (0.5, 21, 42) WER1:  0.2727272727272727 WER2:  0.19480519480519481
0
2
4
6
8
0.8  :  (0.4523809523809524, 19, 42) WER1:  0.2727272727272727 WER2:  0.19480519480519481
0
2
4
6
8
1.0  :  (0.42857142857142855, 18, 42) WER1:  0.2727272727272727 WER2:  0.19480519480519481
0
2
4
6
8
1.2000000000000002  :  (0.3333333333333333, 14, 42) WER1:  0.2727272727272727 WER2:  0.21428571428571427
0
2
4
6
8
1.4000000000000001  :  (0.2857142857142857, 12, 42) WER1:  0.2727272727272727 WER2:  0.22727272727272727


In [43]:
predicted_tokens = []
a = 324
for i,s in enumerate(error_sentences):
    c = autocorrect_word(s, model,l_lambda = 0.4,likelihood = 'bmWithNoLM',prior='transformer',trie = True,top = 4)
#     print(c)
    c = [t[0] for t in c]
    torch.cuda.empty_cache()
    predicted_tokens.append(c)
    if i%2 == 0:
        print(i,WER(correct_tokens[:i+1],predicted_tokens[:i+1] ),word_accuracy(correct_tokens[:i+1],predicted_tokens[:i+1],error_tokens[:i+1]),char_accuracy(correct_tokens[:i+1],predicted_tokens[:i+1],error_tokens[:i+1]))
    if i==323:
        print(WER(correct_tokens[:a],error_tokens[:a] ),WER(correct_tokens[:a],predicted_tokens[:a] ),word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a]),char_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a]))
        break  

0 0.23076923076923078 (0.75, 3, 4) (0.75, 3, 4)
2 0.3076923076923077 (0.5384615384615384, 7, 13) (0.5714285714285714, 8, 14)
4 0.2857142857142857 (0.4166666666666667, 10, 24) (0.4074074074074074, 11, 27)
6 0.23711340206185566 (0.39285714285714285, 11, 28) (0.4375, 14, 32)
8 0.22556390977443608 (0.4594594594594595, 17, 37) (0.5116279069767442, 22, 43)
10 0.19411764705882353 (0.5227272727272727, 23, 44) (0.5384615384615384, 28, 52)
12 0.19270833333333334 (0.5, 25, 50) (0.5333333333333333, 32, 60)
14 0.19369369369369369 (0.5081967213114754, 31, 61) (0.5277777777777778, 38, 72)
16 0.20238095238095238 (0.4857142857142857, 34, 70) (0.5180722891566265, 43, 83)
18 0.20220588235294118 (0.4666666666666667, 35, 75) (0.5111111111111111, 46, 90)
20 0.21283783783783783 (0.4470588235294118, 38, 85) (0.48514851485148514, 49, 101)
22 0.22492401215805471 (0.4148936170212766, 39, 94) (0.44642857142857145, 50, 112)
24 0.22349570200573066 (0.4158415841584158, 42, 101) (0.44166666666666665, 53, 120)
26 0.21

196 0.19218449711723254 (0.4478908188585608, 361, 806) (0.4783068783068783, 452, 945)
198 0.19139512812401138 (0.44987775061124696, 368, 818) (0.48125, 462, 960)
200 0.19026824703680598 (0.4496969696969697, 371, 825) (0.48037190082644626, 465, 968)
202 0.18921417565485363 (0.45269461077844314, 378, 835) (0.4821246169560776, 472, 979)
204 0.18950259383582546 (0.4514218009478673, 381, 844) (0.48129423660262893, 476, 989)
206 0.18991545893719808 (0.4501758499413834, 384, 853) (0.4804804804804805, 480, 999)
208 0.19074626865671643 (0.4491916859122402, 389, 866) (0.47885939036381514, 487, 1017)
210 0.18989958653278205 (0.4526795895096921, 397, 877) (0.48249027237354086, 496, 1028)
212 0.1892919836161498 (0.4542372881355932, 402, 885) (0.48554913294797686, 504, 1038)
214 0.18789808917197454 (0.4574944071588367, 409, 894) (0.4876190476190476, 512, 1050)
216 0.18794835007173602 (0.45805739514348787, 415, 906) (0.4854732895970009, 518, 1067)
218 0.1876065946560546 (0.45901639344262296, 420, 915

# Ablation study -word CD

In [51]:
Gap = 0.2
a = 10
def find_p_lambda(error_sentences):
    
    for j in range(8):
        predicted_tokens = []
        for i,s in enumerate(error_sentences[:a]):
            c = autocorrect_word(s, model,p_lambda = 0,l_lambda =  Gap*j,likelihood = 'defaultWithNoLM',prior='transformer',trie = True,top = 4)
        #     print(c)
            c = [t[0] for t in c]
            predicted_tokens.append(c)
            if i%2 == 0:
                print(i)
#         word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a])
        print(Gap*j,' : ',word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a]),'WER1: ',WER(correct_tokens[:a],error_tokens[:a] ),'WER2: ',WER(correct_tokens[:a],predicted_tokens[:a] ))
find_p_lambda(error_sentences)       

0
2
4
6
8
0.0  :  (0.5238095238095238, 22, 42) WER1:  0.2727272727272727 WER2:  0.4025974025974026
0
2
4
6
8
0.2  :  (0.42857142857142855, 18, 42) WER1:  0.2727272727272727 WER2:  0.3116883116883117
0
2
4
6
8
0.4  :  (0.42857142857142855, 18, 42) WER1:  0.2727272727272727 WER2:  0.3116883116883117
0
2
4
6
8
0.6000000000000001  :  (0.42857142857142855, 18, 42) WER1:  0.2727272727272727 WER2:  0.3116883116883117
0
2
4
6
8
0.8  :  (0.42857142857142855, 18, 42) WER1:  0.2727272727272727 WER2:  0.3116883116883117
0
2
4
6
8
1.0  :  (0.42857142857142855, 18, 42) WER1:  0.2727272727272727 WER2:  0.3116883116883117
0
2
4
6
8
1.2000000000000002  :  (0.23809523809523808, 10, 42) WER1:  0.2727272727272727 WER2:  0.22727272727272727
0
2
4
6
8
1.4000000000000001  :  (0.16666666666666666, 7, 42) WER1:  0.2727272727272727 WER2:  0.24675324675324675


In [52]:
predicted_tokens = []
a = 324
for i,s in enumerate(error_sentences):
    c = autocorrect_word(s, model,l_lambda = 0.4,likelihood = 'defaultWithNoLM',prior='transformer',trie = True,top = 4)
#     print(c)
    c = [t[0] for t in c]
    torch.cuda.empty_cache()
    predicted_tokens.append(c)
    if i%2 == 0:
        print(i,WER(correct_tokens[:i+1],predicted_tokens[:i+1] ),word_accuracy(correct_tokens[:i+1],predicted_tokens[:i+1],error_tokens[:i+1]),char_accuracy(correct_tokens[:i+1],predicted_tokens[:i+1],error_tokens[:i+1]))
    if i==323:
        print(WER(correct_tokens[:a],error_tokens[:a] ),WER(correct_tokens[:a],predicted_tokens[:a] ),word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a]),char_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a]))
        break  

0 0.3076923076923077 (0.75, 3, 4) (0.75, 3, 4)
2 0.46153846153846156 (0.46153846153846156, 6, 13) (0.6428571428571429, 9, 14)
4 0.4 (0.375, 9, 24) (0.48148148148148145, 13, 27)
6 0.35051546391752575 (0.35714285714285715, 10, 28) (0.5, 16, 32)
8 0.3458646616541353 (0.3783783783783784, 14, 37) (0.5581395348837209, 24, 43)
10 0.3176470588235294 (0.4090909090909091, 18, 44) (0.5769230769230769, 30, 52)
12 0.3229166666666667 (0.38, 19, 50) (0.55, 33, 60)
14 0.32882882882882886 (0.3770491803278688, 23, 61) (0.5138888888888888, 37, 72)
16 0.32936507936507936 (0.38571428571428573, 27, 70) (0.5180722891566265, 43, 83)
18 0.3272058823529412 (0.37333333333333335, 28, 75) (0.4888888888888889, 44, 90)
20 0.3310810810810811 (0.3764705882352941, 32, 85) (0.504950495049505, 51, 101)
22 0.3404255319148936 (0.3617021276595745, 34, 94) (0.48214285714285715, 54, 112)
24 0.33810888252148996 (0.36633663366336633, 37, 101) (0.475, 57, 120)
26 0.3271276595744681 (0.37962962962962965, 41, 108) (0.4881889763779

200 0.2975670617592015 (0.37696969696969695, 311, 825) (0.5216942148760331, 505, 968)
202 0.29738058551617874 (0.37604790419161677, 314, 835) (0.5219611848825332, 511, 979)
204 0.297528227036924 (0.3744075829383886, 316, 844) (0.5237613751263903, 518, 989)
206 0.29740338164251207 (0.37514654161781946, 320, 853) (0.5235235235235235, 523, 999)
208 0.29761194029850746 (0.37644341801385683, 326, 866) (0.5231071779744346, 532, 1017)
210 0.2968103957471943 (0.37856328392246297, 332, 877) (0.5233463035019456, 538, 1028)
212 0.29578700994733764 (0.38192090395480227, 338, 885) (0.5260115606936416, 546, 1038)
214 0.294151708164447 (0.38478747203579416, 344, 894) (0.5266666666666666, 553, 1050)
216 0.2938307030129125 (0.38631346578366443, 350, 906) (0.5238987816307404, 559, 1067)
218 0.29363274587834 (0.38688524590163936, 354, 915) (0.5241635687732342, 564, 1076)
220 0.2933408260747401 (0.38503253796095444, 355, 922) (0.5211786372007366, 566, 1086)
222 0.2929883138564274 (0.3849462365591398, 358,

In [31]:
predicted_tokens

[['अब',
  'हाम्रो',
  'छैन',
  'प्रदूषण',
  'नाप्न',
  'भारतीय',
  'पूर्व',
  'प्रधानमन्त्रीलाई',
  'उपप्रधानमन्त्री',
  'निधिले',
  'किन',
  'भेट्ने',
  '।'],
 ['लाठी',
  'लिएका',
  'हजारौँ',
  'संख्यामा',
  'रहेका',
  'थारुहरुले',
  'र्याली',
  'प्रदर्शन',
  'गराई',
  'भोजपुर',
  'बजारको',
  'दक्षिण',
  'चौकमा',
  'कोणसभा',
  'गरेका',
  'थिए',
  '।'],
 ['आफूहरुले',
  'आफ्नो',
  'स्वाभिमानमो',
  'कडा',
  'लडेको',
  'उनको',
  'भनाई',
  'थियो',
  '।'],
 ['प्रदर्शनककरीले',
  'प्रहरी',
  'दमन',
  'बन्द',
  'गर',
  'कैलाली',
  'कञ्चनपुर',
  'छोड्दैनौँ',
  'प्रतिगमनकारी',
  'हतियार',
  'जस्ता',
  'नारा',
  'लगाएका',
  'थिए',
  '।'],
 ['भारतको',
  'अघोषित',
  'नाकाबन्दीको',
  'कारण',
  'पेट्रोलियम',
  'पकदार्थाको',
  'अभाव',
  'भएपछि',
  'सर्वसाधारणले',
  'जोखिमपूर्ण',
  'यात्रा',
  'गर्नुपर्ने',
  'बाध्यता',
  'बढेको',
  'छ',
  '।'],
 ['पेट्रोलियम',
  'पदार्थमा',
  'पाउन',
  'थालेपछि',
  'सिन्धुपाल्चोक',
  'जिल्लाको',
  'बाह्रविसे',
  'चौतारा',
  'मेलम्चीको',
  'सर्वसाधारणले',
  'उनको',
  '

In [22]:
print(i)

122


In [16]:
a = len(predicted_tokens)
WER(correct_tokens[:a],error_tokens[:a] ),WER(correct_tokens[:a],predicted_tokens[:a] )

(0.25023465365121084, 0.16838746010887928)

In [17]:
word_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a])

(0.5993998499624906, 799, 1333)

In [18]:
char_accuracy(correct_tokens[:a],predicted_tokens[:a],error_tokens[:a])

(0.6938255887969447, 1090, 1571)

In [24]:
predicted_tokens

[['अब',
  'गाह्रो',
  'छैन',
  'प्रदूषीण',
  'नाप्न',
  'भारेतीय',
  'पूर्व',
  'ज्रधानमन्त्रीलाई',
  'उपप्रधानकन्त्री',
  'निधीले',
  'किन',
  'भेटे',
  '।'],
 ['लाठी',
  'लिएका',
  'हजारौँ',
  'संख्यामा',
  'रथहेका',
  'थारुहउरुले',
  'र्याली',
  'प्रर्दशन',
  'गरती',
  'नोजापुर',
  'बजिरको',
  'दक्षिण',
  'चौकमा',
  'कोणनसभा',
  'गरेक',
  'थिए',
  '।'],
 ['आफूहरुले',
  'आफनो',
  'स्वाभिमानमो',
  'लडाइ',
  'लडेमो',
  'उनको',
  'भनाई',
  'थियो',
  '।'],
 ['प्रदर्शनककरीले',
  'प्रहरी',
  'दमन',
  'बन्द',
  'गर',
  'कैलाली',
  'कधञ्चनपुर',
  'छोड्दैनौँ',
  'प्रतिगमनकारी',
  'होसियार',
  'जस्ता',
  'नरा',
  'लगाएको',
  'थिए',
  '।'],
 ['भारतको',
  'अघोषित',
  'नाकाबन्अीका',
  'कारण',
  'पेट्रोलियम',
  'पकदार्थाको',
  'अभाव',
  'भएपछि',
  'सीर्ववाधारणले',
  'जोखिमापूर्ण',
  'यात्रा',
  'गर्नुपइर्ने',
  'बाध्यता',
  'वढेको',
  'छ',
  '।'],
 ['पेट्रोलियम',
  'पदार्थ',
  'पाउन',
  'छाडेपछि',
  'सिन्धुपाल्चोक',
  'जिल्लाको',
  'बाह्रविसे',
  'चौतारा',
  'मेलउम्चीका',
  'सर्वसाधारणले',
  'नको'

In [56]:
correct_tokens,error_sentences

([['पुटिनको',
   'आफ्नै',
   'नाइट',
   'उल्भ्स',
   'नामको',
   'बाइक',
   'राइडर',
   'ग्याङ',
   'पनि',
   'छ',
   '।'],
  ['पुटिनको',
   'तर्क',
   'छ',
   'अमेरिका',
   'र',
   'युरोपको',
   'आडमा',
   'कोसोभोले',
   'आत्मनिर्णयको',
   'अधिकार',
   'उपयोग',
   'गर्न',
   'पाउँछ',
   'भने',
   'रुसको',
   'समर्थन',
   'पाएको',
   'क्रिमियाले',
   'नपाउने',
   'कारण',
   'के',
   'छ'],
  ['पुटिनको',
   'विरोधमा',
   'गरिने',
   'सबै',
   'र्',
   'याली',
   'र',
   'सभाहरूमा',
   'यो',
   'गीत',
   'आधिकारिकजस्तै',
   'बनेको',
   'थियो',
   '।'],
  ['पुटिनको',
   'शब्दमा',
   'घुँडा',
   'टेकेर',
   'बसेको',
   'रुस',
   'अब',
   'बिस्तारै',
   'आफ्नो',
   'खुट्टामा',
   'उभिएको',
   'छ',
   '।'],
  ['पुटिन',
   'राष्ट्रपति',
   'भएको',
   'दुई',
   'वर्ष',
   'हुँदा',
   'लेखिएको',
   'गीत',
   'हो',
   'अ',
   'म्यान',
   'लाइक',
   'पुटिन',
   '।'],
  ['पुटिनलाई',
   'गीतमा',
   'देशलाई',
   'सही',
   'दिशामा',
   'डोहोर्',
   'याउने',
   'असल',
   'नेता',
   'उदाहरणीय',
   'व्यक

In [52]:
predicted_tokens

[['पुटिनको',
  'आफ्नो',
  'नाइट',
  'गल्र्स',
  'नामको',
  'बाइक',
  'राइडर',
  'ग्यास',
  'पनि',
  'छ',
  '।'],
 ['पुटिनको',
  'तर्क',
  'छ',
  'अमेरिका',
  'र',
  'युरोपमा',
  'आडमा',
  'कोसोभोले',
  'आत्मनिर्णयको',
  'अधिकार',
  'उपयोग',
  'गर्न',
  'पाउँछ',
  'भने',
  'रुसको',
  'समर्थन',
  'पाएको',
  'इ्रिमियाले',
  'नपाउने',
  'करण',
  'के',
  'छ'],
 ['पुटिनको',
  'विरोधमा',
  'गरिने',
  'सबै',
  'र',
  'गाली',
  'र',
  'सभाहरूमा',
  'को',
  'गीत',
  'आधिकारिकजस्तै',
  'बनेको',
  'थियो',
  '।'],
 ['पटिनको',
  'शब्दमा',
  'घुँडा',
  'टेकेर',
  'बसेको',
  'रुस',
  'अब',
  'बिस्तारै',
  'आफ्नो',
  'खुट्टामा',
  'उभिएको',
  'छ',
  '।'],
 ['पूटिन',
  'राष्ट्रपति',
  'भएको',
  'दुई',
  'वर्ष',
  'हुँदा',
  'लेखिएको',
  'गीत',
  'हो',
  'अ',
  'म्यान',
  'माइक',
  'पुटिन',
  '।'],
 ['पुटिनलाई',
  'गीतमा',
  'देशलाई',
  'सही',
  'दिशामा',
  'फोहोर',
  'पाउने',
  'मसला',
  'नेता',
  'उदाहरणीय',
  'व्यक्तित्व',
  'र',
  'अनुकरणीय',
  'पतिको',
  'रूपमा',
  'देखाइएको',
  'छ',
  '।'],
 ['पूटि

In [41]:
correct_tokens[:20]

[['पुटिनको',
  'आफ्नै',
  'नाइट',
  'उल्भ्स',
  'नामको',
  'बाइक',
  'राइडर',
  'ग्याङ',
  'पनि',
  'छ',
  '।'],
 ['पुटिनको',
  'तर्क',
  'छ',
  'अमेरिका',
  'र',
  'युरोपको',
  'आडमा',
  'कोसोभोले',
  'आत्मनिर्णयको',
  'अधिकार',
  'उपयोग',
  'गर्न',
  'पाउँछ',
  'भने',
  'रुसको',
  'समर्थन',
  'पाएको',
  'क्रिमियाले',
  'नपाउने',
  'कारण',
  'के',
  'छ'],
 ['पुटिनको',
  'विरोधमा',
  'गरिने',
  'सबै',
  'र्',
  'याली',
  'र',
  'सभाहरूमा',
  'यो',
  'गीत',
  'आधिकारिकजस्तै',
  'बनेको',
  'थियो',
  '।'],
 ['पुटिनको',
  'शब्दमा',
  'घुँडा',
  'टेकेर',
  'बसेको',
  'रुस',
  'अब',
  'बिस्तारै',
  'आफ्नो',
  'खुट्टामा',
  'उभिएको',
  'छ',
  '।'],
 ['पुटिन',
  'राष्ट्रपति',
  'भएको',
  'दुई',
  'वर्ष',
  'हुँदा',
  'लेखिएको',
  'गीत',
  'हो',
  'अ',
  'म्यान',
  'लाइक',
  'पुटिन',
  '।'],
 ['पुटिनलाई',
  'गीतमा',
  'देशलाई',
  'सही',
  'दिशामा',
  'डोहोर्',
  'याउने',
  'असल',
  'नेता',
  'उदाहरणीय',
  'व्यक्तित्व',
  'र',
  'अनुकरणीय',
  'पतिका',
  'रूपमा',
  'देखाइएको',
  'छ',
  '।'],
 ['प

In [ ]:
correct_tokens[]

In [61]:
dataset

[('पुटिनको आफ्नै नाइट उल्भ्स नामको बाइक राइडर ग्याङ पनि छ ।',
  'पुटिनको आफ्नै नाइअट उल्भ्स नामको वाइक राइडर ग्याङ पनि छ ।'),
 ('पुटिनको तर्क छ अमेरिका र युरोपको आडमा कोसोभोले आत्मनिर्णयको अधिकार उपयोग गर्न पाउँछ भने रुसको समर्थन पाएको क्रिमियाले नपाउने कारण के छ',
  'पुटिनको तर्क छ अंमेरिका र युरोपको आडमा कोसोभोले आत्मनिर्णयको अधीकोर उपयोग गीर्न पाउँछ भने रुसको समर्थन पाएको इ्रिमियाले नपोउने करिण के छ'),
 ('पुटिनको विरोधमा गरिने सबै र् याली र सभाहरूमा यो गीत आधिकारिकजस्तै बनेको थियो ।',
  'पुटिनको विरोधमा गरिने सबै र् याली र सभाहरूमा यो गीत आधिकारिकजस्तै बनेको थियो ।'),
 ('पुटिनको शब्दमा घुँडा टेकेर बसेको रुस अब बिस्तारै आफ्नो खुट्टामा उभिएको छ ।',
  'पटिनको शब्दमा घुँडा टेकेर बसेको रुस अब बिस्तारै आफ्नो खुट्टाथा उभिएको छ ।'),
 ('पुटिन राष्ट्रपति भएको दुई वर्ष हुँदा लेखिएको गीत हो अ म्यान लाइक पुटिन ।',
  'पूटिन राष्ट्र्पति भएीको दुई वर्ष हुँयदा लेखिएको गीत हो अ म्यान लाइक पुटिन ।'),
 ('पुटिनलाई गीतमा देशलाई सही दिशामा डोहोर् याउने असल नेता उदाहरणीय व्यक्तित्व र अनुकरणीय पतिका रूपमा द